# This section uses Magpie to calculate surface brightness profile

This is equivalently what the previous section does, but magpie is used to calculate surface brightness profile for each Ha Hb line maps and weights, averaged out over the polar angle phi, for each given pixel distance.

what does the package [magpie](https://github.com/knaidoo29/magpie/tree/master) does:

It transforms grid in Cartesian coordinates into (r,phi) polar coordinates, while preserving the size of the surface area of each pixel. Therefore one can calculate surface brightness of a certain ring area by summing over phi of a given radius range. This method is used in [Matharu 2023](https://iopscience.iop.org/article/10.3847/2041-8213/acd1db/pdf) and [Matharu 2024](https://arxiv.org/pdf/2404.17629).

In [7]:
import  magpie              as     magpie
import  numpy               as     np
from    astropy.table       import Table
from    astropy.io          import fits
from    astropy.cosmology   import Planck18
import  astropy.units       as     u
import  matplotlib.pyplot   as     plt
import  matplotlib.colors   as     colors  
from    matplotlib          import use
from    tqdm                import tqdm
from concurrent.futures     import ThreadPoolExecutor, as_completed
import sys, os


#this is just a handy little function to return the desired file path
#give one entry in the object list, return the desired file path
def file_name(obj,prefix,filetype='fits'):
    field = obj['subfield'].lower()
    id    = str(obj['ID']).zfill(5)
    return f"hlsp_clear_hst_wfc3_{field}-{id}_g102-g141_v4_{prefix}.{filetype}"

def find_data(name,hdu):
    for image in hdu:
        if name == image.name:
            return image

obj_lis = Table.read('obj_lis_selected.fits')

In [2]:
import magpie.montecarlo

def spatial_remap(map,pixel_length):
    b2r = magpie.montecarlo.Box2Ring()
    b2r.setup_box(-25*pixel_length,25*pixel_length,50,
                -25*pixel_length,25*pixel_length,50)
    b2r.setup_polar_lin(0., 25*pixel_length, 12, 10, center=[0., 0.])
    b2r.get_weights() 
    return np.linspace(b2r.redges[0], b2r.redges[-1],12), b2r.remap(map)

def radial_profile(obj,linemap,weight,seg,pixel_length):
        linemap = linemap.data; weight = weight.data; seg = seg.data

        seg = np.where(seg==obj['ID'],1,0)
        seg = np.where(linemap>0,seg,0)
        seg = spatial_remap(seg,pixel_length)[1]
        seg_weight = seg/np.sum(seg,axis=0)

        r, linemap_r,   = spatial_remap(np.where(linemap>0,linemap,0),pixel_length)
        linemap_r_wht = spatial_remap(np.where(linemap>0,weight,0),pixel_length)[1]


        map_r   = np.average(linemap_r,weights=seg_weight,axis=0)
        map_r_err = 1/np.sum(linemap_r_wht*seg,axis=0)**0.5
        map_r_std = np.average((linemap_r-map_r)**2,weights = seg_weight,axis=0)**0.5
        map_r_err = (map_r_err**2 + map_r_std**2)**0.5
        return r, map_r, map_r_err 


def gen_radial_table(obj):
    #try:
    path = f"data_extracted/{file_name(obj,prefix='extracted')}"
    with fits.open(path,mode='update') as hdu:
        
        pixel_length = np.deg2rad(find_data('LINE_HA',hdu).header['PIXASEC']/3600) * Planck18.angular_diameter_distance(hdu[1].header['Z_MAP']).to(u.kpc).value

        r, ha_r, ha_r_err = radial_profile(obj,find_data('LINE_HA',hdu),find_data('LINEWHT_HA',hdu),find_data('SEG',hdu),pixel_length)
        r, hb_r, hb_r_err = radial_profile(obj,find_data('LINE_HB',hdu),find_data('LINEWHT_HB',hdu),find_data('SEG',hdu),pixel_length)

        balmer_r     = ha_r/hb_r
        balmer_r_err = ((ha_r_err/hb_r)**2 + (hb_r_err**2 * (ha_r/hb_r**2)**2))**0.5
        
        cols = [
            fits.Column(name='DISTANCE [kpc]',                       format='E', array=r),
            fits.Column(name='Ha_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=ha_r),
            fits.Column(name='Ha_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=ha_r_err),
            fits.Column(name='Hb_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=hb_r),
            fits.Column(name='Hb_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=hb_r_err),
            fits.Column(name='BALMER_DECREM',                        format='E', array=balmer_r),
            fits.Column(name='BALMER_DECREM_ERR',                    format='E', array=balmer_r_err),
        ]
        new_table = fits.BinTableHDU.from_columns(cols, name='RAD_PROFILE')
        for i,image in enumerate(hdu):
            if  image.name == 'RAD_PROFILE':
                hdu[i] = new_table
                hdu.flush()
                print(ha_r)
                return f"{obj['subfield']}-{obj['ID']} processed"

        hdu.append(new_table)
        hdu.flush()
        print(ha_r)
        return f"{obj['subfield']}-{obj['ID']} processed"
    #except Exception as e:
    #        return f"! {obj['subfield']}-{obj['ID']} failed, error{e}"


def cat_process(obj_lis,max_threads=1):
        results = []
        if max_threads > 1 :
            with ThreadPoolExecutor(max_threads) as executor:
                futures = {executor.submit(gen_radial_table,obj):obj for obj in obj_lis}
                for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                    results.append(future.result())
            return results
        else:
            for obj in obj_lis:
                results.append(gen_radial_table(obj))
            return results
def main():

    obj_lis = Table.read('obj_lis_selected.fits')
    results = cat_process(obj_lis,max_threads=6)

    number = 0
    for result in results:
        if 'error' in result:
            number +=1
            print(result)
    print(results)
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)


if __name__ == '__main__':
    main()


0.8578072705548977
Calculating weights : |##################################################| 100% 
Remapping : |##################################################| 100% 
Calculating weights : |##################################################| 100% 
Remapping : |##################################################| 100% 
Calculating weights : |#########################_________________________|  50% 

/tmp/ipykernel_8032/3323356986.py:17: RuntimeWarning: invalid value encountered in divide
  seg_weight = seg/np.sum(seg,axis=0)


Calculating weights : |##################################################| 100% 
Remapping : |##################################################| 100% 
Calculating weights : |##################################################| 100% 
Remapping : |##################################################| 100% 
Calculating weights : |##############################____________________|  60% 

/tmp/ipykernel_8032/3323356986.py:24: RuntimeWarning: divide by zero encountered in divide
  map_r_err = 1/np.sum(linemap_r_wht*seg,axis=0)**0.5


Calculating weights : |##################################################| 100% 
Remapping : |##################################################| 100% 
Calculating weights : |##################################################| 100% 
Remapping : |##################################################| 100% 
[1.99400529 0.75158002 0.16778985 0.0775304  0.0746686  0.06364398
 0.04268063 0.04809547 0.04428592        nan        nan        nan]
0.8269988889942779
Calculating weights : |##################################################| 100% 
Remapping : |##################################################| 100% 
Calculating weights : |##################################################| 100% 
Remapping : |##################################################| 100% 
Calculating weights : |##################################################| 100% 
Remapping : |##################################################| 100% 
Calculating weights : |##################################################| 100% 
Remap

In [9]:
%matplotlib inline
def plot_balmer_decrem(obj):
    try:
        path = f"data_extracted/{file_name(obj,prefix='extracted')}"
        with fits.open(path) as hdu:    
            
            pixel_length = np.deg2rad(find_data('LINE_HA',hdu).header['PIXASEC']/3600) * Planck18.angular_diameter_distance(hdu[1].header['Z_MAP']).to(u.kpc).value
        
            ax = plt.figure(figsize=(8,10));i=1

            for index in [3,2,4,6]:
                ax.add_subplot(int(f'32{i}'));i+=1
                plt.imshow(hdu[index].data,#np.where(seg,hdu[index].data,0),
                            norm=colors.Normalize(vmin=0),
                            origin='lower',
                            cmap = 'plasma_r')
                plt.plot([3,7],[4,4])
                plt.text(5,5,f'{round(pixel_length*4,2)}kpc')
                plt.title(hdu[index].name)
                if index != 2: plt.colorbar()
            
            r,ha_r,ha_r_err, hb_r, hb_r_err, balmer_r, balmer_r_err = np.vstack(find_data('RAD_PROFILE',hdu).data).transpose()            
            ax.add_subplot(325)
            plt.errorbar(r,ha_r,yerr = ha_r_err,fmt='bo:',label='Ha')
            plt.errorbar(r,hb_r,yerr = hb_r_err,fmt='go:',label='Hb')
            plt.xlabel('distance [kpc]'); plt.ylabel('flux [1e-7 erg/s/cm2]')
            plt.yscale('log');plt.grid()
            
            ax.add_subplot(326)
            plt.errorbar(r,balmer_r,yerr = balmer_r_err,fmt='ro:')
            plt.xlabel('distance [kpc]');plt.ylabel('Ha/Hb')
            plt.xlim(0,8);plt.ylim(-5,10)
            plt.grid()
            plt.savefig(f"radial_balmer_decrem/{obj['subfield']}-{obj['ID']}.png")
            plt.savefig(f"sn>10/{obj['subfield']}-{obj['ID']}.png")
            plt.close('all')
            return f"{obj['subfield']}-{obj['ID']}saved"
        

    except Exception as e:
            return f"! {obj['subfield']}-{obj['ID']} failed, error{e}"

def cat_process(obj_lis,max_threads=1):
        results = []
        if max_threads>1:
            with ThreadPoolExecutor(max_threads) as executor:
                futures = {executor.submit(plot_balmer_decrem,obj):obj for obj in obj_lis}
                for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                    results.append(future.result())
            return results
        else:
            for obj in obj_lis:
                results.append(plot_balmer_decrem(obj))
            return results


if __name__ == '__main__':
    use('Agg')
    obj_lis = Table.read('obj_lis_selected.fits')
    results = cat_process(obj_lis,max_threads=1)
    number = 0
    for result in results:
        if 'error' in result:
            number +=1
            print(result)
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)

    

total number of obj processed: 35
number of failed obj 0
